<a href="https://colab.research.google.com/github/AndresMontesDeOca/RegresionAvanzada/blob/main/EjerciciosClase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicios Sueltos dados en Clase

In [1]:
# Autor: Andres Montes de Oca

# Instalacion de Paquetes
!pip install pingouin # No incluido en Google Colab
# !pip install scipy
# !pip install statsmodels


# Cargamos Librerias y Datos
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pingouin as pg
from scipy import stats as st
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tools.tools as smt

# Ignorar Warnings
import warnings
warnings.filterwarnings('ignore')

# Asthetics
sns.set(style='ticks', context='notebook', palette='colorblind', font_scale=0.8, color_codes=True)


# Version rpy2 que no tiene problemas de compatibilidad
!pip install rpy2==3.5.1

# activate R magic
%load_ext rpy2.ipython

# Import rpy2 for dataframe conversion
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
from rpy2.robjects import globalenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7029 sha256=b3aaa158996f9142a7527c482b0eaab4d6b1a08d16499bea5b6336dcf92eeb12
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc03cf7379889266
Successfully built littleutils
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.7/201.7 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rpy2: filename=rpy2-3.5.1-cp310-cp310-linux_x86_64.whl size=318088 sha256=ee6262f413b9b5fa7d58629430a08be5cb99237fb9bf27fda0ce2a3b4e37e147
  Stored in directory: /root/.cache/pip/wheels/73/a6/ff/4e75dd1ce1cfa2b9a670cbccf6a1e41c553199e9b25f05d953
Successfully built rpy2
  Attempting uninstall: rpy2
    Found existing installation: rpy2 3.5.5
    Uninstalling rpy2-3.5.5:
      Successfully uninstalled rpy2-3.5.5


# Carga de Datasets

In [65]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

################################# Datasets ###################################
# Grasa de Cerdos
id = '153lGVzdixcHT-keKg8qmvaoWdPHg6_tB'
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('grasacerdos.xlsx')
dataG = pd.read_excel('grasacerdos.xlsx', index_col='Obs') # Cargamos el Dataset
dataG = dataG.replace(to_replace=',', value='.', regex=True) # Reemplazo , por .
dataG = dataG.astype('float') # Transformo en float

######################################################
# Peso, Edad, Colesterol
id = '17Dv1WcWlc9ojWa6bnSfD7TCRaVKCKqj0'
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('peso_edad_colest.xlsx')
dataP = pd.read_excel('peso_edad_colest.xlsx')
%R -i dataP
######################################################
# Cars con Modelo Lineal
id = '15pf-6P4Ek2rp6mYmfOybHRqAxqOoTp6i'
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('cars.csv')
dataC = pd.read_csv('cars.csv')
%R -i dataC
model_cars = smf.ols('dist ~ speed', data=dataC).fit()
######################################################
# University
id = '16kLQNXhyweAi38xR2IeQCCagVcU725UZ'
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('University.csv')
dataU = pd.read_csv('University.csv')
######################################################
# Iris
dataI = sns.load_dataset('iris')
%R -i dataI
######################################################
# Gorriones
id = '15G6jIOMiuWaTs-qy7eUXwhED4delBMl6'
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('gorriones.xlsx')
dataO = pd.read_excel('gorriones.xlsx')
dataO.rename(columns=
            {'largototal':'Largo', 'extension ':'Alas', 'cabeza':'Cabeza',
             'humero':'Pata', 'esternon':'Cuerpo', 'sobrevida ':'Target'},
            inplace=True)
%R -i dataO
######################################################
# Madera
id = '17F8XFNpCszGHFfs7CpXzZotfmWmXXhoi'
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('madera.csv')
dataM = pd.read_csv('madera.csv').drop(columns='Unnamed: 0')
%R -i dataM

######################################################
# Duncan
id = '17aCADG_APoFTsCdPvdI5T25gBAUE8U2T'
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('Duncan.csv')
dataD = pd.read_csv('Duncan.csv')
%R -i dataD

######################################################
# Infants
id = '17qgVDqEcWeevZh2dHlvPIQ4ueAcTzSjU'
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('low_birth_weight_infants.xlsx')
dataF = pd.read_excel('low_birth_weight_infants.xlsx')
%R -i dataF

######################################################
# MT Cars
id = '17vmBKWIJUt3uiry6WH5-Tyu3ab_mVdX8'
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('mtcars.csv')
dataA = pd.read_csv('mtcars.csv')
dataA.drop(columns='model', inplace=True)
%R -i dataA

######################################################
# Fingerprints
id = '17zjDDk7VZ1fb_6A-UOaDGpzLUyNeG5AW'
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('fingerprints.csv')
dataFP = pd.read_csv('fingerprints.csv').drop(columns='Unnamed: 0')
%R -i dataFP

#### Instalacion de Librerias para R en Google Colab

In [3]:
%%R

# ## MVN Henze-Zirkler Test ### -> Lento
# # system(sudo apt install libgsl-dev)
# install.packages('MVN')

# ## QQ Plot, Bonferroni, outlierTest, vif ### -> Lento
# install.packages('car')

## Wald Test ### -> Rapido
# install.packages('aod')

# LRM Tests ### -> Rapido
# install.packages('lmtest')

## Box-Cox Transformation ### -> Rapido
# install.packages('MASS') # Huber, LTS

## Estimaciones Robustas ### ->
# install.packages('robustbase') # lmrob
# install.packages('olsrr')
# install.packages('quantreg') # LAD

## Regularizacion ### -> Lento
# install.packages('glmnet')



## Seleccion de Variables ### -> Rapido
# install.packages('leaps') # -> regsubsets

NULL


### Clase #2

In [4]:
# print(dataO.describe(), '\n')
# %R print(summary(dataO))

In [5]:
%%R

# model_colest_R = lm('colest ~ edad', data=dataP)
# residuos <- model_colest_R$residuals
# print(summary(model_colest_R))
# print(anova(model_colest_R))

# Bandas de Prediccion
# predichos <- predict(object=model_colest_R, interval='prediction', level=0.95)

# Test de Normalidad adicionales
# install.packages('nortest')
# library(nortest)
# lillie.test(residuos)
# install.packages('car')
# library(car)
# qqPlot(residuos, pch=19)

# Autocorrelacion de Residuos
# install.packages('lmtest')
# library(lmtest)
# dwtest(model_colest_R, alternative='two.sided', iterations=1000)

# Deteccion Outliers
# library(car)
# qqPlot(residuos)
# print(outlierTest(model_colest_R))
# influenceIndexPlot(model_colest_R, vars='Bonf')

NULL


In [6]:
# # No Outliers tests found in Python
%%R

# library(car)

# # Outliers - Bonferroni
# qqPlot(residuos)
# outlierTest(model_colest_R)
# influenceIndexPlot(model_colest_R, vars='Bonf')

# Influyentes - Leverage
# crit_leverage <- 3*mean(hatvalues(model_colest_R))
# leverage <- hatvalues(model_colest_R) > crit_leverage
# data_leverage = cbind(hatvalues(model_colest_R), crit_leverage, leverage)

# # Influyentes - Cook
# influenceIndexPlot(model_colest_R, vars='Cook')

# Influyentes - DFFITS (revisar indices)
# p <- length(model_colest_R$coefficients)
# n <- NROW(model_colest_R$residuals)
# dffits_crit = 2*sqrt(p/n)
# dffits = dffits(model_colest_R)
# data_dffits = data.frame(obs=names(dffits), dffits=dffits)

NULL


In [7]:
# # Exporto a Python y grafico
# %R -o data_dffits
# %R -o dffits_crit

# data_dffits.head()
# sns.scatterplot(data=data_dffits, x=data_dffits.index, y='dffits')
# plt.axhline(y=dffits_crit, linestyle='--', color='red')
# plt.axhline(y=-dffits_crit, linestyle='--', color='red')
# plt.show()

In [8]:
# Analisis analitico del modelo
%%R

# print(head(dataM))
# mod_mad = lm('resist ~ madera', data=dataM)
# print(summary(mod_mad))
# coef = mod_mad$coef
# resid = mod_mad$resid

NULL


In [9]:
# Analisis grafico
# %R -o coef
# print(coef)
# sns.regplot(data=dataM, x='madera', y='resist')
# plt.show()

# plt.plot(dataM.madera, dataM.madera*coef[1]+coef[0])
# plt.ylim(0)
# sns.scatterplotw(data=dataM, x='madera', y='resist')
# plt.show()

In [10]:
# # Analisis de Residuo

# %R -o resid

# # resid = pd.Series(resid, name='Residuo')
# print(pg.normality(resid))
# # print(st.anderson(resid))

# # pg.qqplot(resid)
# # %R qqPlot(resid)

# # Analsis grafico a mano usando Matplotlib
# plt.scatter(x=dataM.index, y=resid)
# plt.xlabel( 'Index')
# plt.ylabel('Residuo')
# plt.title('Correlacion de Residuos')
# plt.axhline(color='grey', linestyle='dashed', alpha=0.5)
# plt.show()
# # No se observa estructura

# # Durbin-Watson Test: 2=No Correlacion (Independecia?)| 0=Correlacion Pos | 4=Correlacion Neg
# print('Durbin-Watson:', sm.stats.durbin_watson(resid)) # Sin validacion Estadistica

In [11]:
# %%R

# # print(dwtest(mod_mad, alternative='two.sided', iterations=1000)) #H0 No Auto-Correlacion
# # print(outlierTest(mod_mad))
# # influenceIndexPlot(mod_mad, vars='Bonf')

# # # Influyentes - Cook
# influenceIndexPlot(mod_mad, vars='Cook')


# # # Influyentes - Leverage
# crit_leverage <- 3*mean(hatvalues(mod_mad))
# leverage <- hatvalues(mod_mad) > crit_leverage
# print(sum(leverage))
# # print(cbind(hatvalues(mod_mad), crit_leverage, leverage))


# # Influyentes - DFFITS (revisar indices en plot)
# p = length(mod_mad$coefficients)
# n = NROW(mod_mad$residuals)
# dffits_crit = 2*sqrt(p/n)
# dffits = dffits(mod_mad)
# data_dffits = data.frame(dffits=dffits)
# print(data_dffits)


In [12]:
# # Influyentes - DFBetas
# %%R

# dfbetas_crit <- 1 # Umbral estandard
# dfbetas(mod_mad)[,2]>dfbetas_crit

In [13]:
# # Exporto a Python y grafico
# %R -o data_dffits
# %R -o dffits_crit

# data_dffits.head()
# sns.scatterplot(data=data_dffits, x=data_dffits.index, y='dffits')
# plt.axhline(y=dffits_crit, linestyle='--', color='red')
# plt.axhline(y=-dffits_crit, linestyle='--', color='red')

In [14]:
# %%R

# predicted = mod_mad$fitted.values
# resid = mod_mad$resid

# library(lmtest)
# bptest(mod_mad)

In [15]:
# library(lmtest)
# dwtest(model_colest_R, alternative='two.sided', iterations=1000) #H0 No Auto-Correlacion

In [16]:
# # Homocedasticidad
# # Importo a Python
# %R -o predicted
# %R -o resid

# # Analsis grafico a mano usando Matplotlib
# plt.scatter(x=predicted, y=resid)
# plt.xlabel( 'Prediccion')
# plt.ylabel('Residuo')
# plt.title('Distribucion de Residuos')
# plt.axhline(color='grey', linestyle='dashed', alpha=0.5)
# plt.show()
# # No se observa estructura de embudo

# # # Analitico (C Test) -> H0: Residuos Homocedasticos
# # BP, p_value, _, _ =sm.stats.het_breuschpagan(model_colest.resid, model_colest.model.exog)
# # print('Estadistico BP y p-value:', BP, p_value)

In [17]:
# # Buevo modelo con Variable transformada
# %%R

# madera2 = dataM$madera**2
# dataM2 = data.frame(dataM, madera2)

# mod_mad2 = lm('resist ~ madera + madera2', data=dataM2)
# print(summary(mod_mad2))

In [18]:
# %R -o dataM2

# dataM2 = sm.add_constant(dataM2)
# mod_mad2 = smf.ols('resist ~ madera + madera2', data=dataM2).fit()

# mod_mad2.summary()


In [19]:
# # Wald Test -> Significatividad de cada Variable
# %%R

# # install.packages('aod')
# # library(aod)

# # # Generamos el Modelo
# # model_gorr = lm('Largo ~ Alas + Cabeza + Pata + Cuerpo', data=dataO)
# # print(summary(model_gorr))

# # # Wald Test para indentificar variables significativas
# wald.test(Sigma = vcov(mod_mad2), b = coef(mod_mad2), Terms = 1) # Cuerpo| H0: Coef = 0 (Var no significativa)


In [20]:
# # Box Cox Transformation

# %%R

# # install.packages('MASS')
# # library(MASS)

# mod_cars <- lm('dist ~ speed', data=dataC)
# # print(summary(mod_cars))
# print(shapiro.test(mod_cars$resid))

# boxcox(object = mod_cars, plotit=TRUE)

# mod_cars2 <- lm('dist**0.5 ~ speed', data=dataC)
# print(summary(mod_cars2))
# print(shapiro.test(mod_cars2$resid))

In [21]:
# # Cuadrados Minimos Ponderados (WLS) en R
# # CAMBIAR EL DATASET DE EJEMPLO
# %%R

# # Generamos el modelo y verificamos la Heterocedasticidad de los residuos analiticamente
# ols_inspec <- lm('inspectores ~ choferes', data=dataCh)
# print(bptest(ols_inspec))

# # Confirmamos con el analisys grafico en Python

# # Generamos la ponderacion de pesos y modelamos de nuevo
# pesos = I(1/ols_onspec$fitted.values**2) # A los valores mas alejados se le reduce el peso
# wls_inspec <- lm('inspectores ~ choferes', data=dataCh, weights=pesos)
# print(summary(wls_inspec))

### Clase 3

In [22]:
# # Filtro el DF
# data = dataU[['nassets', 'stfees']]

# display(data.head())

# # Importo a R
# %R -i data

# # Ploteamos la Regresion
# sns.regplot(data=data, x='stfees', y='nassets')
# plt.show()

In [23]:
# %%R

# model = lm('nassets ~ stfees', data=data)
# resid = model$resid
# predicted = predict(model)
# fitted = model$fitted.values

# # Testeamos Normalidad de Residuos
# print(shapiro.test(resid))
# # qqPlot(resid)
# # Residuos no Normales

In [24]:
# # Analsis grafico de Homosedasticidad

# # Importo Datos desde R
# %R -o resid
# %R -o predicted
# %R -o fitted

# plt.scatter(x=predicted, y=resid)
# plt.xlabel( 'Prediccion')
# plt.ylabel('Residuo')
# plt.title('Distribucion de Residuos')
# plt.axhline(color='grey', linestyle='dashed', alpha=0.5)
# plt.show()
# # Si se observa estructura de embudo

In [25]:
# %%R
# library(lmtest)

# # Test de Homocedasitcidad
# print(bptest(model))

# # Test de Auto-Correlacion
# print(dwtest(model, alternative='two.sided', iterations=1000)) #H0 No Auto-Correlacion

In [26]:
# %%R
# print(model)

In [27]:
# # display(data.head())

# nassets_log10 = pd.Series(np.log10(data['nassets']), name='nassets_log10')
# stfees_log10 = pd.Series(np.log10(data['stfees']), name='stfees_log10')
# nassets_logE = pd.Series(np.log(data['nassets']), name='nassets_logE')
# stfees_logE = pd.Series(np.log(data['stfees']), name='stfees_logE')


# data = pd.concat([data, nassets_log10, stfees_log10, nassets_logE, stfees_logE], axis=1)

# # Exporto a R
# %R -i data

In [28]:
# # Transformacion Box Cox
# %%R

# library(MASS)

# # Buscamos el Landa optimo -> Si Lambda  = 0 -> log10(y) | y**Lambda
# # boxcox(object = model_2, plotit=TRUE)

# # Generamos el nuevo modelo con la variable dependiente transformada, y verificamos su nueva normalidad
# model_2 <- lm('nassets_log10 ~ stfees_log10', data=data)

# # Normalidad
# print(shapiro.test(model_2$resid))

# # Test de Homocedasitcidad
# print(bptest(model_2))

# # Test de Auto-Correlacion
# print(dwtest(model_2, alternative='two.sided', iterations=1000)) #H0 No Auto-Correlacion
# # No olvidarse de anti-transformar en los IC, IP y Fit

In [29]:
# # Bandas de Prediccion

# %R predichos <- predict(object=model_2, interval='prediction', level=0.95)
# %R nuevos_datos <- data.frame(predichos, data)

# %R -o nuevos_datos

# display(nuevos_datos.head())

# # Grafico
# sns.regplot(data=nuevos_datos, x='stfees_log10', y='nassets_log10')
# sns.lineplot(data=nuevos_datos, x='stfees_log10', y='lwr', linestyle='--', color='r')
# sns.lineplot(data=nuevos_datos, x='stfees_log10', y='upr', linestyle='--', color='r')
# plt.show()


In [30]:
# valores = [10000]

# newdata = pd.Series(np.log10(valores), name='stfees_log10').to_frame()
# %R -i newdata
# display(newdata)



In [31]:
# %%R
# print(class(newdata))

# predict(object=model_2, newdata=newdata)


### Clase 3: Teoria

In [32]:
# data = dataD
# %R -i data
# data.head()

In [33]:
# sns.pairplot(data=data)
# plt.show()

# display(sns.heatmap(data.corr(), vmin=-1, vmax=1, cmap='RdYlGn', annot=True)) # Todos contra todos
# plt.show()

In [34]:

# %%R

# model <- lm('income ~ education', data=data)
# resid <- model$resid
# fitted <- model$fitted.values
# print(summary(model))

In [35]:
# # LRM en Python

# data = sm.add_constant(data)
# model = smf.ols('income ~ education', data=data).fit()
# print(model.summary())

In [36]:
# sns.regplot(data=data, x='education', y='income')
# plt.show()

In [37]:
# %%R

# print(shapiro.test(resid))

In [38]:
# # Analisis Analitico (Breusch-Pagan Test) -> R | H0: Residuos Homocedasticos
# %%R

# library(lmtest)
# bptest(model)

In [39]:

# # # Analsis Grafico -> Python

# # Importamos desde R
# %R -o resid
# %R -o fitted

# plt.scatter(x=fitted, y=resid)
# plt.xlabel( 'Prediccion')
# plt.ylabel('Residuo')
# plt.title('Distribucion de Residuos')
# plt.axhline(color='grey', linestyle='dashed', alpha=0.5)
# plt.show()
# # No se observa estructura de embudo

# # # Analisis Analitico Python
# # BP, p_value, _, _ =sm.stats.het_breuschpagan(model.resid, model.model.exog)
# # print('Estadistico BP y p-value:', BP, p_value)

In [40]:
# %%R
# # Outliers - Bonferroni

# library(car)

# qqPlot(resid)
# outlierTest(model)
# influenceIndexPlot(model, vars='Bonf')

In [41]:
# data.query('index == 15 or index ==26 or index ==5')

In [42]:
# # Influyentes - Cook
# %%R

# # print(cooks.distance(model))
# influenceIndexPlot(model, vars='Cook')

In [43]:
# %%R
# # Influyentes - DFFITS
# p <- length(model$coef)
# n <- NROW(model$resid)
# dffits_crit <- 2*sqrt(p/n)
# dffits <- dffits(model)
# data_dffits <- data.frame(dffits=dffits)

In [44]:
# # DFFITS Grafico -> Python
# %R -o data_dffits
# %R -o dffits_crit

# # Grafico
# data_dffits.head()
# sns.scatterplot(data=data_dffits, x=data_dffits.index, y='dffits')
# plt.axhline(y=dffits_crit, linestyle='--', color='red')
# plt.axhline(y=-dffits_crit, linestyle='--', color='red')
# plt.xticks(rotation=45)
# plt.show()

# # Filtro los criticos
# dffits_crit_low = -dffits_crit
# dffits_crit_up = dffits_crit

# display(data_dffits.query('dffits > @dffits_crit_up or dffits < @dffits_crit_low'))

In [45]:
# %%R
# print(summary(model))

In [46]:
%%R

## Estimaciones Robustas ### ->
# install.packages('robustbase')
# library(robustbase)
# install.packages('olsrr')
# library(olsrr)
# install.packages('quantreg')
# library(quantreg)


NULL


In [47]:
# data = dataF

# data.head()

In [48]:

# # Linear Regression
# %%R
# data <- dataF

# model <- lm('headcirc ~ gestage + birthwt', data=data)
# resid <- model$resid
# fitted <- model$fitted.values
# print(summary(model))


In [49]:
# # LRM en Python

# # Generamos el Modelo
# data = sm.add_constant(data)
# model = smf.ols('headcirc ~ gestage + birthwt', data=data).fit()
# print(model.summary())

In [50]:
# data.head()

In [51]:
# %%R
# library(robustbase)

# model_rob <- lmrob(headcirc ~ ., data=data)
# print(summary(model_rob))

In [55]:
# # Multicolinealidad
# %%R

# data <- dataA
# head(data)

In [56]:
# # # Multicollonearity - Eeach independent variable should be independent from other independent variables
# data = dataA


# display(sns.heatmap(data.corr(), vmin=-1, vmax=1, cmap='RdYlGn', annot=True)) # Todos contra todos
# plt.show()

In [57]:
# %%R

# data <- dataA
# library(car)


# model1 <- lm(mpg ~ drat + disp + hp + cyl, data=data)
# # print(summary(model1))
# vif1 <- vif(model1)
# print(vif1)

# model2 <- lm(mpg ~ drat + disp + hp, data=data)
# # print(summary(model2))
# vif2 <- vif(model2)
# print(vif2)


In [58]:
# # LRM en Python

# # Generamos el Modelo
# data = sm.add_constant(data)
# model = smf.ols('mpg ~ drat + disp + hp', data=data).fit()
# # print(model.summary())

In [59]:
# # Metodos de Regularizacion

# %%R
# # set.seed(100)

# # Train Test Slit
# train_idx <- sample(1:nrow(data), size=.75*(nrow(data)))
# train <- data[train_idx,]
# test <- data[-train_idx,]

# # Modelamos
# model_comp <- lm(mpg ~ ., data=train)
# print(summary(model_comp))
# # print(coef(model_comp))

In [60]:
# # Caluclo MSE
# %%R
# pred <- predict(model_comp, data=test)
# model_mse <- mean((pred - test$mpg)**2)
# print(model_mse)

In [61]:
# # Regularizacion
# %%R
# library(glmnet)
# # set.seed(100)

# X = model.matrix(mpg ~ .-1, data=data)
# y = data$mpg
# std <- TRUE

# #################### Ridge
# # Valor optimo para Lambda con 10-fold CV (alpha=0 Ridge)
# ridge_cv = cv.glmnet(X, y, alpha=0, type='mse', family='gaussian', standarize=std, nfolds=10)

# # Ajustamos Ridge con 100 opciones de valores para lambda
# ridge_mdl = glmnet(X, y, alpha=0, nlambda=100, standardize=std)

# # Extraemos los coeficientes
# ridge_coef <- coef(ridge_mdl, s=ridge_cv$lambda.min)
# # print(ridge_cv$lambda.min)
# # print(ridge_coef)


# ################## Laso
# # Valor optimo para Lambda con 10-fold CV ( alpha=1 Lasso)
# lasso_cv = cv.glmnet(X, y, alpha=1, type='mse', family='gaussian', standarize=std, nfolds=10)

# # Ajustamos Lasso con 100 opciones de valores para lambda
# lasso_mdl = glmnet(X, y, alpha=1, nlambda=100, standardize=std)

# # Extraemos los coeficientes
# lasso_coef <- coef(lasso_mdl, s=lasso_cv$lambda.min)
# # print(lasso_cv$lambda.min)
# # print(lasso_coef)


# ################## Elastic Net
# # Valor optimo para Lambda con 10-fold CV ( alpha=0.5 ElasticNet)
# enet_cv = cv.glmnet(X, y, alpha=0.5, type='mse', family='gaussian', standarize=std, nfolds=10)

# # Ajustamos Lasso con 100 opciones de valores para lambda
# enet_mdl = glmnet(X, y, alpha=0.5, nlambda=100, standardize=std)

# # Extraemos los coeficientes
# enet_coef <- coef(enet_mdl, s=enet_cv$lambda.min)
# # print(enet_cv$lambda.min)
# # print(enet_coef)

In [62]:
# # Comparamos los modelos con Regularizacion
# %%R

# # Nu se
# tmatrix <- model.matrix(mpg ~ ., test)[,-1]

# # Ridge
# ridge_pred <- predict(ridge_mdl, s=ridge_cv$lambda.min, newx=tmatrix)
# ridge_mse <- mean((ridge_pred-test$mpg)**2)

# # Lasso
# lasso_pred <- predict(lasso_mdl, s=lasso_cv$lambda.min, newx=tmatrix)
# lasso_mse <- mean((lasso_pred-test$mpg)**2)

# # Elastic Net
# enet_pred <- predict(enet_mdl, s=enet_cv$lambda.min, newx=tmatrix)
# enet_mse <- mean((enet_pred-test$mpg)**2)

# # Generamos Dataframe con los MSE
# MSEs <- data.frame(ridge_mse, lasso_mse, enet_mse, model_mse)
# print(MSEs)

In [68]:
# Reduccion de la Dimensionalidad
data = dataFP

display(data.head())
print(data.shape)

,X1,X2,X3,X4,X5,X6,X11,X12,X15,X16,...,X792,X793,X795,X798,X800,X801,X805,X806,X812,X813
0,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


(165, 388)
